In [ ]:
# Step 1 Install Needed Dependencies and Define Constants

# Dependencies
!pip3 install pandas
!pip3 install openpyxl
!pip3 install requests
import requests
import pandas as pd
import io
import json

# Constants
USERNAME = '<YOUR_USERNAME>'
PASSWORD = '<YOUR_PASSWORD>'
TOKEN_URL='https://identity.vinter.co/authenticate/login'
SYMBOL_LONG='vnfa-weti-15-d'
RECORD_LIMIT=50
SYMBOL_SHORT='WETI'
LIST_FILES_URL=f'https://identity.vinter.co/organization/files/{SYMBOL_SHORT}'


In [ ]:
# Step 2: Use the username and password to get an access token that will be used to read PCF information.

def get_access_token():
  formdata = {
      "username": USERNAME,
      "password": PASSWORD
  }
  headers = {
      "Content-Type": "application/x-www-form-urlencoded"
  }
  response = requests.post(TOKEN_URL, data=formdata, headers=headers)
  response_json = response.json()
  return response_json['access_token']

token = get_access_token()

print("Your access token is: " + token)

# Define the headers that will be used to make requests to the PCF API.
headers = {
  "authorization": f"Bearer {token}"
}

In [ ]:
# Step 3: Get a List of Your Organization Files

def get_organization_files():
  response = requests.get(LIST_FILES_URL, headers=headers)
  return response.json()
  
files = get_organization_files()

# filter out files whose 'folder' is not 'pcf'
files = [file for file in files if file['folder'] == 'pcf']
print(f"Got {len(files)} files for {SYMBOL_SHORT}.")

In [ ]:
# Define a map for storing aggregate data
aggregate_data = {}

# Go through the files one by one
count = 0
for file in files:
  count += 1
  print(f"Processing file {count}/{len(files)}")
  GET_FILE_PRESIGNED_URL=f'https://identity.vinter.co/organization/files/{SYMBOL_SHORT}/pcf/{file["label"]}/{file["name"]}'
  url = requests.get(GET_FILE_PRESIGNED_URL, headers=headers).json()
  xls_data = requests.get(url).content
  bytes_io_wrapped = io.BytesIO(xls_data)

  df = pd.read_excel(
    bytes_io_wrapped, 
    engine='openpyxl',
    sheet_name='Portfolio Composition'
  )

  # Save Price USD, Ticker, Amount, Symbol, Current Weight to the map
  aggregate_data[file["label"]] = json.loads(df[['Price USD', 'Ticker', 'Amount', 'Current Weight']].to_json(orient='records'))

In [ ]:
# Save the aggregate data as a convenient json file

SAVED_FILE_PATH = f'{SYMBOL_SHORT}_aggregate_data.json'

with open(SAVED_FILE_PATH, 'w') as f:
  f.write(json.dumps(aggregate_data, indent=2))

print("Your aggregate data has been saved to: " + SAVED_FILE_PATH)